In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import warnings
import itertools
import statsmodels.api as sm
import sklearn
import numpy as np
import seaborn as sns
from google.colab import files
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.utils import check_array
from sklearn import tree
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_iris
from math import sqrt
from sklearn.model_selection import train_test_split
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose 
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from subprocess import check_call
from subprocess import call
from IPython.display import Image
!pip install graphviz


from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def return_type(df,type):

  if type == "num":
    return df.select_dtypes(include='float64')
  elif type == "cate":
    return df.select_dtypes(include = 'object')
  else:
    print("num or cate")

def datadrop(df,list,column):
  new_list = df.drop(list, axis=column)
  return new_list


In [ ]:
print ('loading data')
train = pd.read_csv('/content/gdrive/MyDrive/Humana/train_post_drop.csv',index_col=0)


loading data


In [ ]:
new_type = pd.read_csv('/content/gdrive/MyDrive/Humana/new_type.csv')

In [ ]:
cat = new_type["0"][0:66]
num = new_type["1"]

In [ ]:
train[cat] = train[cat].astype("object")

In [ ]:
train.dtypes.value_counts()

float64    201
object      66
dtype: int64

**Feature Engineering: creating interaction between race and age group**


In [ ]:
#"20-35":1,"35-65":2,"65-80":3,"80-105":4
bin=[20,35,65,80,105]
binlabel=["1","2","3","4"]
train["age_cutoff"] = pd.cut(train["est_age"],bins=bin,labels=binlabel,include_lowest=True)

In [ ]:
train["race_cd"] = train["race_cd"].replace(np.nan,0)
train["race_cd"] = train["race_cd"].astype("int64")

In [ ]:
def valuation_formula(x, y):
    return str(x)+str(y)

train["race_age"] = train.apply(lambda row: valuation_formula(row['race_cd'], row['age_cutoff']), axis=1)

In [ ]:
train["race_age"].unique()

array(['13', '12', '14', '04', '52', '02', '03', '53', '23', '24', '22',
       '21', '11', '33', '43', '34', '62', '42', '63', '32', '31', '44',
       '01', '54', '51', '61', '64', '41'], dtype=object)

**Numerical missing value imputation**

In [ ]:
#M1:mean

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
train_num_m1 = imp.fit_transform(train[num])
train_num_m1 = pd.DataFrame(train_num_m1,index=None,columns = train[num].columns)


In [ ]:
train_num_m1

,auth_3mth_post_acute_dia,rx_gpi2_72_pmpm_cost_6to9m_b4,atlas_pct_laccess_child15,atlas_recfacpth14,atlas_pct_fmrkt_frveg16,atlas_pct_free_lunch14,bh_ip_snf_net_paid_pmpm_cost_9to12m_b4,auth_3mth_acute_ckd,bh_ncal_pmpm_ct,bh_ip_snf_net_paid_pmpm_cost_3to6m_b4,cons_chmi,auth_3mth_post_acute_trm,auth_3mth_post_acute_rsk,rx_gpi4_6110_pmpm_ct,atlas_pc_snapben15,credit_bal_nonmtgcredit_60dpd,rx_bh_mbr_resp_pmpm_cost_9to12m_b4,atlas_pct_laccess_nhna15,credit_hh_nonmtgcredit_60dpd,rx_bh_pmpm_ct_0to3m_b4,cons_lwcm10,atlas_fsrpth14,atlas_wicspth12,cmsd2_sns_digest_abdomen_pmpm_ct,atlas_ghveg_farms12,credit_hh_bankcardcredit_60dpd,total_outpatient_allowed_pmpm_cost_6to9m_b4,cons_cwht,atlas_netmigrationrate1016,atlas_pct_laccess_snap15,rx_nonmaint_mbr_resp_pmpm_cost_9to12m_b4,atlas_naturalchangerate1016,ccsp_236_pct,auth_3mth_post_acute_ben,atlas_pct_laccess_hisp15,auth_3mth_dc_no_ref,rx_overall_mbr_resp_pmpm_cost,rx_overall_gpi_pmpm_ct_0to3m_b4,atlas_pct_laccess_hhnv15,credit_bal_consumerfinance,...,bh_physician_office_copay_pmpm_cost_6to9m_b4,rwjf_income_inequ_ratio,rej_total_physician_office_visit_ct_pmpm_0to3m_b4,credit_num_nonmtgcredit_60dpd,credit_bal_autofinance_new,rwjf_men_hlth_prov_ratio,rx_gpi2_56_dist_gpi6_pmpm_ct_3to6m_b4,cmsd2_sns_genitourinary_pmpm_ct,bh_ncal_pct,atlas_pct_snap16,ccsp_227_pct,atlas_ghveg_sqftpth12,rx_days_since_last_script_6to9m_b4,atlas_orchard_acrespth12,atlas_pct_laccess_multir15,cons_cgqs,ccsp_065_pmpm_ct,atlas_medhhinc,rwjf_mental_distress_pct,atlas_pct_laccess_nhpi15,credit_num_consumerfinance_new,rx_gpi2_49_pmpm_cost_0to3m_b4,cons_chva,atlas_avghhsize,rx_overall_net_paid_pmpm_cost_6to9m_b4,atlas_ownhomepct,atlas_orchard_farms12,atlas_pct_fmrkt_wic16,rx_gpi2_33_pmpm_ct_0to3m_b4,rwjf_social_associate_rate,atlas_freshveg_farms12,auth_3mth_post_acute_inf,days_since_last_clm_0to3m_b4,mcc_end_pct,cons_lwcm07,atlas_pct_fmrkt_otherfood16,atlas_pct_laccess_white15,auth_3mth_post_acute_mean_los,rx_gpi2_66_pmpm_ct,cons_ltmedicr
0,0.0,0.000000,7.910346,0.049413,0.000000,46.839102,0.0,0.0,0.0,0.0,67.000000,0.0,0.0,0.0,8.068443,2366.225593,0.00,0.036550,22.614359,0.0,0.267160,0.904069,0.248279,0.0,5.0,9.803185,0.00,70.000000,9.775188,1.458254,0.000000,2.255148,0.0,0.0,1.190413,0.0,14.424166,2.000000,1.150322,832.960852,...,0.0,3.621404,0.000000,0.621714,1389.427219,0.000400,0.000000,0.0,0.0,12.083352,0.000000,147.994686,217.0,0.000000,1.260405,88.000000,0.0,56861.0,0.103341,0.000000,0.073287,3.956667,158.00000,2.12,38.320000,73.922397,0.0,0.0,0.333333,23.839718,5.0,0.0,120.0,0.000000,0.233590,0.00,26.842903,0.0,0.000000,4.000000
1,0.0,0.000000,1.730272,0.095624,60.000000,36.721923,0.0,0.0,0.0,0.0,69.000000,0.0,0.0,0.0,12.354272,2926.079093,0.00,0.028851,19.627044,0.0,0.142650,0.721653,0.087810,0.0,6.0,10.081917,0.00,75.000000,9.931234,1.063555,0.000000,3.093653,0.0,0.0,1.251418,0.0,27.267500,3.666667,2.447362,814.885956,...,0.0,4.430692,0.000000,0.549206,1795.154717,0.001456,0.333333,0.0,0.0,8.677720,0.000000,80.357201,120.0,0.156425,0.310503,82.000000,0.0,55828.0,0.106975,0.002699,0.228002,7.793334,254.00000,2.58,0.000000,68.704457,21.0,0.0,0.333333,12.576445,69.0,0.0,120.0,0.000000,0.191980,50.00,11.405128,0.0,0.000000,3.000000
2,0.0,0.000000,5.015501,0.022398,30.000000,39.605015,0.0,0.0,0.0,0.0,76.000000,0.0,0.0,0.0,14.184090,4522.696898,0.17,0.086740,36.273675,0.0,0.259040,0.444301,0.103268,0.0,1.0,15.611655,4192.19,55.000000,5.555760,2.516320,0.000000,3.323222,0.0,0.0,1.715413,0.0,23.571666,1.333333,2.212277,1796.896312,...,0.0,3.682825,0.000000,1.167164,1989.082598,0.000981,0.000000,0.0,0.0,15.466672,1.958872,60.517063,231.0,1.037510,0.773873,98.000000,0.0,61903.0,0.113497,0.000000,0.502758,0.000000,118.00000,2.57,1.186667,68.700726,10.0,0.0,0.000000,9.649039,14.0,0.0,36.0,0.000000,0.441110,0.00,1.713138,0.0,0.083333,5.422812
3,0.0,2.266667,4.049586,0.070407,50.000000,93.694818,0.0,0.0,0.0,0.0,46.000000,0.0,0.0,0.0,25.220000,3993.478363,0.00,0.167049,34.035432,0.0,0.279520,0.696737,0.087032,0.0,0.0,22.73282

In [ ]:
#M2:knn

imputer = KNNImputer() # default n_neighbors is 5
train_m2 = imputer.fit_transform(train)
train_m2 = pd.DataFrame(train_m2,index=None,columns = num)

In [ ]:
train[num].head(30)

,auth_3mth_post_acute_dia,rx_gpi2_72_pmpm_cost_6to9m_b4,atlas_pct_laccess_child15,atlas_recfacpth14,atlas_pct_fmrkt_frveg16,atlas_pct_free_lunch14,bh_ip_snf_net_paid_pmpm_cost_9to12m_b4,auth_3mth_acute_ckd,bh_ncal_pmpm_ct,bh_ip_snf_net_paid_pmpm_cost_3to6m_b4,cons_chmi,auth_3mth_post_acute_trm,auth_3mth_post_acute_rsk,rx_gpi4_6110_pmpm_ct,atlas_pc_snapben15,credit_bal_nonmtgcredit_60dpd,rx_bh_mbr_resp_pmpm_cost_9to12m_b4,atlas_pct_laccess_nhna15,credit_hh_nonmtgcredit_60dpd,rx_bh_pmpm_ct_0to3m_b4,cons_lwcm10,atlas_fsrpth14,atlas_wicspth12,cmsd2_sns_digest_abdomen_pmpm_ct,atlas_ghveg_farms12,credit_hh_bankcardcredit_60dpd,total_outpatient_allowed_pmpm_cost_6to9m_b4,cons_cwht,atlas_netmigrationrate1016,atlas_pct_laccess_snap15,rx_nonmaint_mbr_resp_pmpm_cost_9to12m_b4,atlas_naturalchangerate1016,ccsp_236_pct,auth_3mth_post_acute_ben,atlas_pct_laccess_hisp15,auth_3mth_dc_no_ref,rx_overall_mbr_resp_pmpm_cost,rx_overall_gpi_pmpm_ct_0to3m_b4,atlas_pct_laccess_hhnv15,credit_bal_consumerfinance,...,bh_physician_office_copay_pmpm_cost_6to9m_b4,rwjf_income_inequ_ratio,rej_total_physician_office_visit_ct_pmpm_0to3m_b4,credit_num_nonmtgcredit_60dpd,credit_bal_autofinance_new,rwjf_men_hlth_prov_ratio,rx_gpi2_56_dist_gpi6_pmpm_ct_3to6m_b4,cmsd2_sns_genitourinary_pmpm_ct,bh_ncal_pct,atlas_pct_snap16,ccsp_227_pct,atlas_ghveg_sqftpth12,rx_days_since_last_script_6to9m_b4,atlas_orchard_acrespth12,atlas_pct_laccess_multir15,cons_cgqs,ccsp_065_pmpm_ct,atlas_medhhinc,rwjf_mental_distress_pct,atlas_pct_laccess_nhpi15,credit_num_consumerfinance_new,rx_gpi2_49_pmpm_cost_0to3m_b4,cons_chva,atlas_avghhsize,rx_overall_net_paid_pmpm_cost_6to9m_b4,atlas_ownhomepct,atlas_orchard_farms12,atlas_pct_fmrkt_wic16,rx_gpi2_33_pmpm_ct_0to3m_b4,rwjf_social_associate_rate,atlas_freshveg_farms12,auth_3mth_post_acute_inf,days_since_last_clm_0to3m_b4,mcc_end_pct,cons_lwcm07,atlas_pct_fmrkt_otherfood16,atlas_pct_laccess_white15,auth_3mth_post_acute_mean_los,rx_gpi2_66_pmpm_ct,cons_ltmedicr
0,0.0,0.000000,7.910346,0.049413,0.000000,46.839102,0.0,0.0,0.0,0.0,67.0,0.0,0.0,0.0,8.068443,2366.225593,0.000000,0.036550,22.614359,0.000000,0.26716,0.904069,0.248279,0.000000,5.0,9.803185,0.000000,70.0,9.775188,1.458254,0.000000,2.255148,0.000000,0.0,1.190413,0.0,14.424166,2.000000,1.150322,832.960852,...,0.0,3.621404,0.0,0.621714,1389.427219,0.000400,0.000000,0.000000,0.0,12.083352,0.000000,147.994686,217.0,0.000000,1.260405,88.0,0.0,56861.0,0.103341,0.000000,0.073287,3.956667,158.0,2.12,38.320000,73.922397,0.0,0.000000,0.333333,23.839718,5.0,0.0,120.0,0.000000,0.23359,0.000000,26.842903,0.0,0.000000,4.0
1,0.0,0.000000,1.730272,0.095624,60.000000,36.721923,NaN,0.0,0.0,0.0,69.0,0.0,0.0,0.0,12.354272,2926.079093,0.000000,0.028851,19.627044,0.000000,0.14265,0.721653,0.087810,0.000000,6.0,10.081917,0.000000,75.0,9.931234,1.063555,0.000000,3.093653,0.000000,0.0,1.251418,0.0,27.267500,3.666667,2.447362,814.885956,...,0.0,4.430692,0.0,0.549206,1795.154717,0.001456,0.333333,0.000000,0.0,8.677720,0.000000,80.357201,120.0,0.156425,0.310503,82.0,0.0,55828.0,0.106975,0.002699,0.228002,7.793334,254.0,2.58,0.000000,68.704457,21.0,0.000000,0.333333,12.576445,69.0,0.0,120.0,0.000000,0.19198,50.000000,11.405128,0.0,0.000000,3.0
2,0.0,0.000000,5.015501,0.022398,30.000000,39.605015,0.0,0.0,0.0,0.0,76.0,0.0,0.0,0.0,14.184090,4522.696898,0.170000,0.086740,36.273675,0.000000,0.25904,0.444301,0.103268,0.000000,1.0,15.611655,4192.190000,55.0,5.555760,2.516320,0.000000,3.323222,0.000000,0.0,1.715413,0.0,23.571666,1.333333,2.212277,1796.896312,...,0.0,3.682825,0.0,1.167164,1989.082598,0.000981,0.000000,0.000000,0.0,15.466672,1.958872,NaN,231.0,1.037510,0.773873,98.0,0.0,61903.0,0.113497,0.000000,0.502758,0.000000,118.0,2.57,1.186667,68.700726,10.0,0.000000,0.000000,9.649039,14.0,0.0,36.0,0.000000,0.44111,0.000000,1.713138,0.0,0.083333,NaN
3,0.0,2.266667,4.049586,0.070407,50.000000,93.694818,0.0,0.0,0.0,0.0,46.0,0.0,0.0,0.0,25.220000,3993.478363,0.000000,0.167049,34.035432,0.000000,0.27952,0.696737,0.087032,0

In [ ]:
# MinMax Scaler: transform data into 0~1
# Notice that is has been reported that standardization N~(0, 1) is more accurate on mean squared error
scaling=MinMaxScaler()
scaled_train_xxx = pd.DataFrame(scaling.fit_transform(train_num_m1),columns = train[num].columns)

In [ ]:
train.dtypes.value_counts()

float64     201
object       66
int64         1
category      1
dtype: int64

In [ ]:
# Standard Scaler
from sklearn.preprocessing import StandardScaler

train_std_scaled = train.copy()

# apply standardization on numerical features
for i in num:
    
    # fit on training data column
    scale = StandardScaler().fit(train[[i]])
    
    # transform the training data column
    train_std_scaled[i] = scale.transform(train[[i]])

In [ ]:
train_post_scaling = train_std_scaled.copy()

In [ ]:
train_post_scaling.dtypes.value_counts()

float64    201
object      67
dtype: int64

In [ ]:
train_post_scaling = train_post_scaling.drop("age_cutoff",axis = 1)

In [ ]:
train_post_scaling[cat] = train_post_scaling[cat].astype("object")
train_post_scaling[num] = train_post_scaling[num].astype("float64")

In [ ]:
train_post_scaling["race_age"].head()

0    13
1    13
2    12
3    13
4    13
Name: race_age, dtype: object

In [ ]:
train_post_scaling.to_csv('train_post_scaling.csv')

In [ ]:
!cp train_post_scaling.csv "/content/gdrive/MyDrive/Humana/train_post_scaling.csv"